### 题目
实现第一个神经网络，一个隐藏层。
要求
* 实现具有单个隐藏层的2类分类神经网络
* 使用具有非线性激活功能的单位，例如tanh
* 计算交叉熵损失
* 实现向前和向后传播

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from load_dataset import load_planar_dataset

### 获取数据集
获取数据集的API已经有不需要实现，数据有两个类别标签y = 0，y = 1的点组成，每个点有两个特征。建立一个模型来拟合这个数据。

<img src="images/数据集.png" style="width:600px;height:300px;">

In [2]:
X, Y = load_planar_dataset()

In [3]:
print ('数据集特征值的形状:', X.shape)
print ('数据集目标值的:', Y.shape)
print ('样本数:', X.shape[1])

数据集特征值的形状: (2, 400)
数据集目标值的: (1, 400)
样本数: 400


In [4]:
def sigmoid(x):
    """
    """
    s = 1/(1+np.exp(-x))
    return s

### 单隐层神经网络模型
<img src="images/classification_kiank.png" style="width:600px;height:300px;">
* 前向传播
$$Z^{[1]} = W^{[1]}X+b^{[1]}$$                       

$$形状：(4,m) = (4,2) * (2,m) + (4,1)$$

$${A}^{[1]}=\sigma(Z^{[1]})$$                        
$$形状：(4,m)$$

$$Z^{[2]} = W^{[2]}A^{[1]}+b^{[2]}$$                 
$$形状：(1,m) = (1,4) * (4,m)+(1,1)$$

$$A^{[2]}=\sigma(Z^{[2]})$$                          
$$形状：(1,m)$$

$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

 计算损失：
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

* 步骤
    * 定义网络结构
    * 初始化参数
    * 循环一下步骤
        * 前向传播
        * 计算损失
        * 反向传播获得梯度
        * 梯度更新

### 1、定义神经网络结构
网络输入输出以及隐藏层神经元个数

In [5]:
def layer_sizes(X, Y):
    """
    """
    # 输入层大小
    n_x = X.shape[0]
    # 隐层大小
    n_h = 4
    # 输出层大小
    n_y = Y.shape[0]
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

### 2、 初始化模型参数
随机初始化权重以及偏置为0

In [6]:
def initialize_parameters(n_x, n_h, n_y):
    """
    输入每层的神经元数量
    
    返回：隐层、输出层的参数
    """
    
    np.random.seed(2)
    
    ### 开始
    # 创建隐层的两个参数
    # 让值小一些
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    
    # 创建输出层前对应的参数
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    ### 结束
    
    # 检测维度是否符合要求
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 3、循环中的第一步：前向传播
根据之前给的前向传播公式，完成该函数
$$Z^{[1]} = W^{[1]}X+b^{[1]}$$

$${A}^{[1]}=tanh(Z^{[1]})$$             

$$Z^{[2]} = W^{[2]}A^{[1]}+b^{[2]}$$

$$A^{[2]}=\sigma(Z^{[2]})$$            

使用的函数：np.dot,np.tanh, np.sigmoid

In [7]:
def forward_propagation(X, parameters):
    """
    Argument:
    X:(n_feature, m)
    
    Returns:
    A2:最后一层的输出
    cache:用于反向传播计算的存储中间计算结果的字典
    """
    # 获取参数
    ### 开始
    # 取出每一层的参数
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # 进行一层一层的运算
    Z1 = np.matmul(W1, X) + b1
    A1 = np.tanh(Z1)
    
    # 第二层
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    ### 结束
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

### 3、循环中的二步：计算损失
完成损失计算的过程，根据损失公式
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

设计多个维度，使用np.multiply进行乘法运算

In [8]:
def compute_cost(A2, Y, parameters):
    """
    parameters：最后一层输出，目标值，参数
    return:损失
    """
    m = Y.shape[1]
    ### 开始
    logpro = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), (np.log(1 - A2)))
    cost = - 1 / m * np.sum(logpro)
    
    ### 结束
    
    cost = np.squeeze(cost)
    
    assert(isinstance(cost, float))
    
    return cost

### 3、循环中的第三步：反向传播
反向传播在这个网络中分为两步
第一次计算：
$$dZ^{[2]} = A^{[2]} - Y$$

$$dW^{[2]}=\frac{1}{m}dZ^{[2]}{A^{[1]}}^{T}$$

$$db^{[2]}=\frac{1}{m}np.sum(dZ^{[2]}, axis=1)$$

隐层的反向传播梯度计算：
$$dZ^{[1]} = {W^{[2]}}^{T}dZ^{[2]}*{(1-g(Z^{[1]})}^{2}={W^{[2]}}^{T}dZ^{[2]}*{(1-A^{[1]})}^{2}$$

$$dW^{[1]}=\frac{1}{m}dZ^{[1]}X^{T}$$

$$db^{[1]} = \frac{1}{m}np.sum(dZ^{[1]}, axis=1)$$



In [9]:
def backward_propagation(parameters, cache, X, Y):
    """
    parameters：
    cache：存储每层前向传播计算结果
    X：数据特征
    Y：数据目标值
    
    return:每个参数的梯度
    """
    # 得出训练样本数量
    m = X.shape[1]
    
    # 获取参数和缓存中的输出
    ### 开始
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
    ### 结束
    
    # 反向传播计算梯度
    ### 开始
    # 最后一层的参数梯度计算
    dZ2 = A2 - Y
    dW2 = 1/m * np.dot(dZ2, A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    
    # 隐藏层的参数梯度计算
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
    ### 结束
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

### 3、循环中的第四步：更新梯度

In [14]:
def update_parameters(parameters, grads, learning_rate = 0.005):
    """
    参数：网络参数，梯度，学习率
    返回更新之后的参数
    """
    # 获取参数以及梯度
    ### 开始
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    ## 结束
    
    # 使用学习率更新参数
    ### 开始
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    ### 结束
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 4、建立网络模型训练逻辑

In [15]:
def nn_model(X, Y, num_iterations = 10000, print_cost=False):
    """
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # 初始化参数
    ### 开始
    # 获取网络的层大小
    # 2, 4, 1
    n_x, n_h, n_y = layer_sizes(X, Y)
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    ### 结束
    
    # 循环
    for i in range(0, num_iterations):
         
        ### 开始
        # 前向传播
        A2, cache = forward_propagation(X, parameters)
        
        # 计算损失
        cost = compute_cost(A2, Y, parameters)
        
        # 反向传播计算梯度
        grads = backward_propagation(parameters, cache, X, Y)
        
        # 利用梯度更新参数
        parameters = update_parameters(parameters, grads)
        
        ### 结束
        
        if i % 1000 == 0:
            print ("迭代次数 %i: %f" %(i, cost))

    return parameters

### 5、预测结果

In [16]:
def predict(parameters, X):
    """
    """
    
    # 计算概率值，以及判断类别
    A2, cache = forward_propagation(X, parameters)
    predictions = np.array( [1 if x >0.5 else 0 for x in A2.reshape(-1,1)] ).reshape(A2.shape)
    
    return predictions

In [20]:
# 测试
parameters = nn_model(X, Y, num_iterations = 10000)

predictions = predict(parameters, X)
print ('准确率: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

迭代次数 0: 0.693048
迭代次数 1000: 0.692359
迭代次数 2000: 0.688928
迭代次数 3000: 0.682109
迭代次数 4000: 0.677535
迭代次数 5000: 0.674585
迭代次数 6000: 0.670325
迭代次数 7000: 0.661691
迭代次数 8000: 0.649615
迭代次数 9000: 0.638593
准确率: 58%
